# Baseline definieren

Für die Erstellung einer Baseline wird ein Klassifikator für die Klassifikationsaufgabe des Beispiel Titanic erstellt, mit dem transformierten Trainingsdatenset trainiert und dem transformierten Validierungsdatenset validiert.

Pakete importieren

In [1]:
import pickle
from sklearn.pipeline import Pipeline

Transformierte Daten laden

In [2]:
with open('../output/titanic/datasets_transformed.pkl', 'rb') as handle:
    datasets_transformed = pickle.load(handle)

## Klassifikator erstellen

Als Klassifikator wird ein Entscheidungsbaum verwendet. Scikit-learn liefert den DecisionTreeClassifier. Es handelt sich um einen Prädiktor. Die fit()-Methode wird mit den transformierten Trainingsdaten aufgerufen, um den Entscheidungsbaum zu erstellen. 

In [3]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(random_state=0)
classifier.fit(datasets_transformed['X_train'], datasets_transformed['y_train'])

DecisionTreeClassifier(random_state=0)

Über den Aufruf der score()-Methode erhält man den Accuracy-Score des Validierungsdatensets.

In [4]:
classifier.score(datasets_transformed['X_val'], datasets_transformed['y_val'])

0.7513812154696132

Das übliche Vorgehen beim Machine Learning ist **experimentell**. Man entwickelt zunächst eine Variante und erhält ein Ergebnis. In unserem Fall ein Accuracy-Score von 0.75. Das bedeutet 75% der vorhergesagten Werte sind richtig. Dieser Score dient als Basis für weitere Optimierungen. Es werden Veränderungen unterschiedlichster Art vorgenommen wie zum Beispiel 
* Anwendung weiterer Transformationsschritte
* Entfernen von Transformationsschritte
* Änderung der Transformationseinstellungen
* Hinzufügen von Merkmalen
* Entfernen von Merkmalen
* Modifizieren von Merkmalen
* Ändern des Machine Learning Algorithmus 
* Ändern der Hyperparameter

Nach **jeder Änderung** wird **geprüft** ob sich das Ergebnis, der Score, **verbessert oder verschlechtert** hat und entsprechend die Änderung beibehalten oder verworfen. Häufig sind es sehr viele Experimente die durchgeführt werden müssen. Es fällt schwer den Überblick zu behalten und es ist aufwendig die Experimente manuell durchzuführen. Bei der Variation von Hyperparameter kann die sogenannte **Grid-Search**[^footnote3] eingesetzt werden um die Experimente zu automatisieren. Für jeden Hyperparameter wird eine begrenzte Menge von möglichen Werten bestimmt, die getestet werden soll. Grid-Search **testet alle Kombinationen und gibt die Wertekombination des besten Ergebnisses aus**.

Wie bereits zu Beginn dieses Abschnitts erwähnt, ist es möglich am Ende der Pipeline statt eines Transformer einen beliebigen Estimator einzusetzen. Ein beliebiger Estimator kann auch ein Prediktor sein. So kann beim Anwendungsbeispiel Titanic einfach der Klassifikator am Ende der Pipeline eingefügt werden. Einer der Vorteile, wenn man die Vorverarbeitungsschritte und den Prediktor in einer Pipeline integriert ist, dass **Grid-Search auch für die Vorverarbeitungsschritte** eingesetzt werden kann.

## Klassifikator in Pipeline integrieren

Erstellt wird eine Pipeline, die im ersten Schritt die bereits erstellte Transformer-Pipeline enthält und im Anschluss den Klassifikator.

```{figure} ../images/pipelineFull.png
---
height: 180px
align: center
name: fig-pipelineFull
---
```

Transformer Pipeline laden

In [5]:
with open('../output/titanic/transformer_pipeline.pkl', 'rb') as handle:
    transformer_pipeline = pickle.load(handle)

Datensets laden

In [6]:
with open('../output/titanic/datasets.pkl', 'rb') as handle:
    datasets = pickle.load(handle)

Pipeline erstellen

In [7]:
full_pipeline = Pipeline(steps=[
    ('transformers', transformer_pipeline),
    ('predictor', DecisionTreeClassifier(random_state=0))
])

Die Pipeline wird mit dem Trainingsdatenset trainiert und dem Validierungsset validiert.

In [8]:
full_pipeline.fit(datasets['X_train'], datasets['y_train'])
full_pipeline.score(datasets['X_val'], datasets['y_val'])

0.7513812154696132

Das Ergebnis ist wie erwartet unverändert. Die Accuracy beträgt 0.75. Nun können mit der Grid Search Methode optimale Einstellungen gefunden werden.